# Text Analytics Lab 1: Regular Expressions and Classifiers

### Learning Outcomes
* Be able to set up a Python and Jupyter notebook environment for text analytics
* Understand how to use regular expressions to preprocess text
* Know how to carry out text normalisation and train and test naïve Bayes and logistic regression classifiers.
* Be able to examine learned model parameters and explain how each classifier makes a decision.

### Outline

1. Getting started: how to set up your environment, Jupyter notebooks introduction
1. Acquiring raw text data
1. Regular expressions
1. Text normalisation 
1. Training and evaluating naïve Bayes using Scikit-learn.
1. Training and evaluating logistic regression using Scikit-learn.
1. Optional extension: lemmatization and bigram features.
1. Optional extensions: lexicon features.

### How To Complete This Lab

Read the text and the code then look for 'TODOs' that instruct you to complete some missing code. Look out for 'QUESTIONS' which you should try to answer before moving on to the next cell. Aim to work through the lab during the scheduled lab hours. To get help, you can talk to TAs or the lecturer during the labs, post questions to the Blackboard discussion board or on Teams, or ask a question in the lectures. 

The labs *will not be marked*. However, they will prepare you for the coursework, so try to keep up with the weekly labs and have fun with the exercises! To understand what's going on inside the methods we use here, make sure to watch the lecture videos for the same week.

## 1. Getting Started

### Setting up your environment

We recommend using ```conda``` to create an environment with the correct versions of all the packages you need for these labs. You can install either Anaconda or Miniconda, which will include the ```conda``` program. 

We provide a .yml file that lists all the packages you will need, and the versions that we have tested the labs with. You can use this file to create your environment as follows.

1. Open a terminal. Use the command line to navigate to the directory containing this notebook and the file ```crossplatform_environment.yml```. You can use the command ```cd``` to change directory on the command line.

1. Run the conda program by typing ```conda env create -f crossplatform_environment.yml```, then answer any questions that appear on the command line.

1. Activate the environment by running the command ```conda activate text_analytics```.

1. Make kernel available in Jupyter: ```python -m ipykernel install --user --name=text_analytics```.

1. Relaunch Jupyter: shutdown any running instances, and then type ```jupyter lab``` into your command line.

1. Find this notebook and open it up again.

1. Go to the top menu and change the kernel: click on 'Kernel'--> 'Change kernel' --> text_analytics.

You should now be ready to go!

The core libraries we will be using in this unit are:

- [Datasets](https://huggingface.co/docs/datasets/), produced by HuggingFace, is a hub for lots of interesting text datasets.
- [NLTK](https://www.nltk.org), a comprehensive NLP library.
- [Scikit-learn](https://scikit-learn.org/stable/user_guide.html), for machine learning and classifier evaluation.
- [Gensim](https://radimrehurek.com/gensim/), for topic modelling.
- [Transformers](https://huggingface.co/docs/transformers/en/index), for state-of-the-art NLP models. 
- [PyTorch](https://pytorch.org/), a framework for deep learning. 

The libraries above have good documentation, which is available either online (links above) or via Python itself, e.g. `help(numpy.array)` in the Python interpreter. 

### Refreshers for Python and Jupyter

**Skip this part if you're already familiar with Python and Jupyter notebooks.**

This lab assumes you have used Python and Jupyter Notebooks before. 

For an introduction or refresher on Python, see the [Introduction to Python lab](https://github.com/UoB-COMS21202/lab_sheets_public/tree/master/lab_1) or the University of Bristol [Beginning Python](https://milliams.gitlab.io/beginning_python/) course. If you are a beginner with Python, you might also like to look at Chapter 1 in the NLTK book, which also provides a guide for "getting started with Python": https://www.nltk.org/book/ 

The labs will be run on [Jupyter Notebook](http://jupyter.org/), an interactive coding environment embedded in a webpage supporting various programing languages (Python, R, Lua, etc.) through the concept of kernels.  

It allows you to enrich your code with complex comments formatted in Markdown and $\LaTeX$, as well as to place the results of your computation right below your code.

Notebooks are organised in cells which can contain either code (in our case, this will be Python code) or text, which can be easily and nicely formatted using the Markdown notation. 

To edit an already existing cell simply double-click on it. You can use the toolbar to insert new cells, edit and delete them (or use keyboard shortcuts which are very handy to speed up coding). 

Cells can be run by hitting `shift+enter` when editing a cell or by clicking on the `Run` button at the top. Running a Markdown cell will simply display the formatted text, while running a code cell will execute the commands executed in it. Create new cells with the keyboard shortcut `esc` followed by `A` or `B`.

**Note**: when you run a code cell, all the created variables, implemented functions and imported libraries will be then available to every other code cell. It is commonly assumed that cells will be run in the correct sequence and running them repeatedly or out-of-order may sometimes cause errors. To reset all variables and functions (for debugging) simply click `Kernel > Restart` from the Jupyter menu.

#### Markdown 

Markdown cells allow you to write fancy and simple comments: all of this is written in Markdown - double click on this cell to see the source. An introduction to Markdown syntax can be found [here](https://daringfireball.net/projects/markdown/syntax).

As Markdown is translated to HTML upon displaying it also allows you to use pure HTML: more details are available [here](https://daringfireball.net/projects/markdown/syntax#html).

Finally, you can also display simple $\LaTeX$ equations in Markdown thanks to `MathJax` support. For inline equations wrap your equation between `$` symbols; for display mode equations use `$$`.

## 1. Acquiring Raw Text Data

Now, let's get some text data! We'll start with the IMDB dataset, which contains movie reviews along with their classification into "positive" or "negative" sentiment. Run the code below to download the data from [HuggingFace's datasets hub](https://huggingface.co/datasets/imdb):

In [1]:
!python -m pip install nltk

In [2]:

!python -m pip install datasets

In [3]:
from datasets import load_dataset
import numpy as np

cache_dir = "./data_cache"

# The data is already divided into training and test sets.
# Load the training set:
train_dataset = load_dataset(
    "imdb", # name of the dataset collection
    split="train",  # train or test
    cache_dir=cache_dir,
)
print(f"Training dataset with {len(train_dataset)} instances loaded")

train_dataset = np.random.choice(train_dataset, 5000, replace=False)  # we'll only use a subset of the data in this lab so that the code runs quicker


Training dataset with 25000 instances loaded


We can access the documents in the dataset like elements in a list. For example, document 3 looks like this:

In [4]:
train_dataset[3]

{'text': "Considering the risk of showing same-sex relationships before the late 1980's, Personal Best could have done better to play the same-sex relationship between Hemingway (Chris Cahill) and Donnelly (Tory Skinner) as a more than experimental phase of Cahill's life.<br /><br />It seems to me that the creators of this movie threw in the same-sex relationship between two fairly attractive women in order to attract viewers. Also consider the 90 seconds of exposing the crotches of several women jumping backwards over a high jump pole. This random scene had VERY LITTLE relevance to the movie and it appeared as though this was done merely to keep the audience interested in this bland movie. I suppose the producers were trying to counteract the boring plot and the even more boring setting of the movie (the 1980 Oregon Track and Field Competition).<br /><br />This review may seem harsh, but it is the truth. The exploitation of young Muriel Hemingway's body and the same-sex relationship r

In [5]:
train_dataset[10]

{'text': 'Edwin Porter\'s 1903 short film entitled "The Great Train Robbery" bursts onto the screen with so much excitement and ingenuity that one prepares to be blown away by another pioneering early film. Just like Melies\' "A Trip to the Moon", critics have hailed this as being the film that introduced the western genre into modern cinema. In my eyes, they were right. It had everything from the planning, the actual heist of the train, the murder of an unwanted civilian, and that looming final scene that makes you realize that these villains mean business \x96 it was all monumental for its time. From here to Eastwood, every western filmmaker has used Porter\'s image in some form or another to create their own story. One cannot say that this film didn\'t open the door, but the struggle comes from the story itself. The genre was defined by Porter, but outside of its initial excitement \x96 there really isn\'t anything to grapple onto. Perhaps I am jaded by the cliché modern westerns an

# 2. Regular Expressions

## 2.1 Search

We'll start by trying out some regular expressions. Suppose we want to identify tweets where people discuss really loved about certain movies. We could start by looking for tweets that contain the word 'love'. A first step would be to find all occurrences of the word 'love'. Review the code below to see how we can do this:

In [6]:
import re  # Python regular expressions library

all_matches = []

for review in train_dataset:
    matches = re.findall('love', review['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

1784
1784
love


This has given us a list of matches in the variable `all_matches`, which all contain the string 'love', but not the sentences themselves.
This isn't very useful, but we can do better if we define the right regular expression!

Regular expressions define a pattern, rather than a specific string, allowing us to generalise our search and retrieve a many different strings that match the pattern.
In Python, we differentiate a regular expression from a normal string by putting an 'r' character in front of the string.

We can generalise our search by using a _disjunction_, which will match against any one of a set of characters. The disjunction is written inside square brackets. 

Let's try to retrieve instances of the word `"love" followed by any letter`. We can write a disjunction that matches any lower case letter as `[a-z]`:

In [7]:
all_matches = []

for review in train_dataset:
    matches = re.findall(r'love [a-z]', review['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

926
926
love v
love m
love l
love w
love a
love c
love n
love s
love g
love h
love o
love p
love u
love t
love f
love k
love d
love i
love e
love j
love r
love b
love y


Our current search only matches a single letter of the word after 'love'. The length of that following word is variable, so how can we write an expression to match the whole word? 

Here, we can use a special character, '\*', which will match against zero or more repetitions of the preceding regular expression. Let's try it out:

In [8]:
all_matches = []

for review in train_dataset:
    matches = re.findall(r'love [a-z]*', review['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

1018
1018
love one
love scifi
love on
love animals
love cinema
love step
love already
love hurts
love could
love making
love both
love between
love crucifies
love hidden
love each
love his
love cars
love because
love of
love surfing
love does
love all
love them
love such
love it
love shifts
love in
love musicals
love him
love obsession
love bad
love 
love thing
love her
love horror
love their
love after
love feelings
love that
love undying
love cheesy
love where
love these
love along
love affair
love not
love since
love kenny
love action
love you
love dark
love twist
love really
love dance
love never
love triangle
love anyways
love the
love those
love doubtfully
love amateur
love many
love throughout
love is
love bravery
love theme
love story
love has
love at
love almost
love toward
love splatter
love when
love me
love being
love every
love child
love jazz
love song
love twisted
love most
love secret
love mystery
love episode
love video
love songs
love like
love well
love more
love rom

Let's say we only want to retrieve the word following 'love', not the string containing 'love ' itself. 
We can do this using parentheses to create _groups_ of characters, such as this: `([a-z]*)`. The resulting matches will be returned as tuples of groups, and any characters not inside parentheses will not be returned as part of any group. Try out the code below to see this, and note that the space character after 'love' is not returned in the matches.

In [9]:
all_matches = []

for review in train_dataset:
    matches = re.findall(r'(love) ([a-z]*)', review['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

for match in set(all_matches):  # just print the following
    print(match[1]) 

1018
1018
('love', 'before')
('love', 'since')
('love', 'movies')
('love', 'hip')
('love', 'of')
('love', 'animals')
('love', 'surfing')
('love', 'has')
('love', 'what')
('love', 'in')
('love', 'obsession')
('love', 'virtual')
('love', 'being')
('love', 'anyways')
('love', 'never')
('love', 'because')
('love', 'mystery')
('love', 'any')
('love', 'such')
('love', 'who')
('love', 'expressed')
('love', 'at')
('love', 'someone')
('love', 'like')
('love', 'shifts')
('love', 'is')
('love', 'but')
('love', 'hidden')
('love', 'affair')
('love', 'romance')
('love', 'still')
('love', 'song')
('love', 'apple')
('love', 'things')
('love', 'between')
('love', 'with')
('love', 'more')
('love', 'secret')
('love', 'each')
('love', 'most')
('love', 'crappy')
('love', 'when')
('love', 'horses')
('love', 'trying')
('love', 'these')
('love', 'without')
('love', 'biangle')
('love', 'many')
('love', 'triangle')
('love', 'about')
('love', 'bravery')
('love', 'cult')
('love', 'beyond')
('love', 'his')
('love'

Now, let's try to retrieve the preceding words as well. It would be better to match capital letters as well as lower case, which we can do with the disjunction `[a-zA-Z]`. 

TODO 1: complete the code below to retrieve only the words that precede and follow 'love', including capitalised and lower case words.

In [10]:
all_matches = []

for review in train_dataset:
    
    ### WRITE YOUR CODE HERE
    matches = re.findall(r'([a-zA-Z]*) love', review['text'])
    ########
    
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

1654
1654

another
probably
noir
or
art
sweet
exemplifies
depressed
standing
so
castle
anime
about
everyone
animation
audiences
aquarium
masochistic
animal
crossed
must
find
Pecker
your
abiding
person
finding
mortal
The
doomed
boy
genitals
showing
making
serious
potential
ll
be
first
no
Biblical
greatest
however
little
German
incestuous
movie
lady
film
la
d
awkward
Soon
innate
simple
one
transport
her
both
Great
funniest
show
around
wife
usually
tragic
passionate
Fuller
wanting
u
what
Monitors
secretly
a
time
every
they
hop
untouched
two
fashioned
unpretentious
Berlin
wholly
corny
see
was
still
and
leads
simply
doctor
that
Cedric
daughter
gay
other
dead
past
saif
my
forbidden
romantic
Action
who
sexy
Mitch
horse
through
talking
nightmarish
He
by
the
those
very
free
sports
fictional
platonic
I
should
also
Many
IMDb
i
all
been
have
requisite
more
lost
sure
he
future
Academy
race
Davis
unfinished
like
interesting
horror
well
new
variety
My
male
you
is
course
ethereally
pointless
Luciano
o

In [11]:
 re.findall(r'[^\.\!\?]* love [^\.\!\?]*', review['text'])

[]

This is starting to look more useful, but we still want to retrieve whole sentences. 

Sentences in English are usually demarcated by punctuation, so let's use the following punctuation marks to identify sentence boundaries: `'.', '!', '?'`. Those punctuation marks are special characters when used in regular expressions, so to force Python to interpret them literally, we need to put the escape character '\\' in front of them. 

Now, we can write a disjunction that matches against the punctuation like this: `[\.\!\?]`.

So far, we have assumed the text consists only of letters. Can you think of any characters we have excluded here? 

A better way to find all matches would be to use _negation_ to match against any character _except_ the punctuation marks that bound the sentences. A negation will match any character except those specified, which we can write like this: `[^\.\!\?]`, where the '^' indicates the negation.


TODO 2: Retrieve whole sentences containing 'love'. To do this, modify our previous expression by using negation to match all of the characters except '.', '!', and '?'.

In [56]:
all_matches = []

for review in train_dataset:
    
    ### WRITE YOUR CODE HERE
    matches = re.findall(r'[^\.\!\?]* love [^\.\!\?]*', review['text'])
    ########
    
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match)  

962
<br /><br />Rambo returns and declares that he intends to drift aimlessly until "this country loves us as much as we love it," bringing up another myth that Vets returning from Vietnam were uniformly spat upon and cursed, which is why I guess we elected so many to the Congress and appointed others to high-echelon positions
 Is safe sex realistic in all instances of love and lust between passionate and emotional human beings
<br /><br />I love the way the story builds up slowly at first, then gradually escalates
) fall in love with each other when they share their souls and minds in letters passed through PO Box 237
 That's where he meets Sarah (Neve Campbell) and he falls in love with her
 The movie is a sweet reminder that love can be found just anywhere if your antennae is acutely active
 In what should have been best picture of the year, we see how lives are easily broken when the love of money becomes the ultimate pursuit in order to ease our troubled lives
 John Ritter ineptly

Look at the results -- does the regular expression correctly return sentences containing 'love'?

There are lots more special characters that you can use to form really powerful regular expressions for segmenting, retrieving and substituting text. For your reference, you can find a complete list [here](https://docs.python.org/3/library/re.html#regular-expression-syntax).

## 2.2 Substitution

We can also use regular expressions during _preprocessing_ to clean up text and prepare it for further analysis. For this, we use regular expression _substitution_, which finds a matching string within a larger piece of text, and replaces it with another string.

Let's use this to clean up the text by removing the line break characters.

In Python, we can use the re.sub() function, which takes three arguments:
1. The expression to match. 
2. The pattern we should replace it with
3. The text to apply the subtitution to. 

Some of the reviews contain some HTML formatting code, `<br />`, which we can try to remove to clean up the text. We can do this by writing an expression for the first argument of re.sub() that matches '<br />'. Take a look at how this works by running the code below:

In [13]:
print('ORIGINAL TEXT: ')
print(train_dataset[3]['text'])
    
clean_article = re.sub(r'<br />', r' ', train_dataset[3]['text'])  # replace HTML breaks with a space
    
print('CLEANER TEXT: ')
print(clean_article)

ORIGINAL TEXT: 
Considering the risk of showing same-sex relationships before the late 1980's, Personal Best could have done better to play the same-sex relationship between Hemingway (Chris Cahill) and Donnelly (Tory Skinner) as a more than experimental phase of Cahill's life.<br /><br />It seems to me that the creators of this movie threw in the same-sex relationship between two fairly attractive women in order to attract viewers. Also consider the 90 seconds of exposing the crotches of several women jumping backwards over a high jump pole. This random scene had VERY LITTLE relevance to the movie and it appeared as though this was done merely to keep the audience interested in this bland movie. I suppose the producers were trying to counteract the boring plot and the even more boring setting of the movie (the 1980 Oregon Track and Field Competition).<br /><br />This review may seem harsh, but it is the truth. The exploitation of young Muriel Hemingway's body and the same-sex relation

# 3. Text Normalisation 

For most text analytics tasks, we will first need to transform the raw text to a suitable format for input to method such as a classifier. This process is called _text normalisation_ and is part of the _preprocessing_ stage. There are three common steps:

1. Sentence segmentation: we have already tried out a basic approach to obtaining complete sentences using regular expressions. This would need to be modified to return a list of all sentences in a document. 
2. Tokenisation, in which the sentences are split into a sequence of tokens, which include words, numbers and punctuation marks.
3. Word normalisation, in which different forms of a word are replaced by a root form.

We are now going to see how to perform these steps using the NLTK library.

## 3.1 Sentence Segmentation

Let's start by using NLTK to split a document into sentences. This should give better results than our regular expressions above.

You may get some errors from NLTK when you try to use sent_tokenize or word_tokenize further down. This is usually because you need to download and install some NLTK data. Please check the error message to find out which package is required. You probably need to install packages called 'punkt' and 'wordnet'. You can install these packages by running the cell below.

In [14]:
import nltk 

#nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Ha Tung
[nltk_data]     Lam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
import nltk
nltk.download('punkt_tab') 

review = train_dataset[0]['text']

# tokenize the sentence
sents = nltk.sent_tokenize(review)
print(len(sents))

for sent in sents[:5]:
    print("<SENTENCE>")
    print(sent)  # print the first five sentences of this document

8
<SENTENCE>
Remembering the dirty particulars of this insidiously vapid "movie" is akin to digging into your chest cavity with a rusty, salted spoon.
<SENTENCE>
Perhaps "Home Alone 2: Lost in New York" (1992) was a bit on the predictable side, but this pathetic excuse for a film is just one of the most shameless bids at commercialization I have ever heard of.
<SENTENCE>
A boy fighting off spies/terrorists when he's home alone in a Chicago suburb with the chickenpox?
<SENTENCE>
Ridiculous!
<SENTENCE>
Why did this film have to be made?


[nltk_data] Downloading package punkt_tab to C:\Users\Ha Tung
[nltk_data]     Lam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


TODO 3: Use the regular expression substitution code from section 2.2 to remove the '\<br /\>' tags from the sentences displayed above and print the results.

In [16]:
clean_sents = []

for sent in sents[:5]:
    
    ### WRITE YOUR OWN CODE HERE
    sent = re.sub(r'<br />', r' ', sent)
    #######
    
    print("<SENTENCE>")
    print(sent)  # print the first five sentences of this document
    
    clean_sents.append(sent)  # save the cleaned sentences for later

<SENTENCE>
Remembering the dirty particulars of this insidiously vapid "movie" is akin to digging into your chest cavity with a rusty, salted spoon.
<SENTENCE>
Perhaps "Home Alone 2: Lost in New York" (1992) was a bit on the predictable side, but this pathetic excuse for a film is just one of the most shameless bids at commercialization I have ever heard of.
<SENTENCE>
A boy fighting off spies/terrorists when he's home alone in a Chicago suburb with the chickenpox?
<SENTENCE>
Ridiculous!
<SENTENCE>
Why did this film have to be made?


In [17]:
clean_sents

['Remembering the dirty particulars of this insidiously vapid "movie" is akin to digging into your chest cavity with a rusty, salted spoon.',
 'Perhaps "Home Alone 2: Lost in New York" (1992) was a bit on the predictable side, but this pathetic excuse for a film is just one of the most shameless bids at commercialization I have ever heard of.',
 "A boy fighting off spies/terrorists when he's home alone in a Chicago suburb with the chickenpox?",
 'Ridiculous!',
 'Why did this film have to be made?']

## 3.2 Tokenisation

NLTK provides a similar function for tokenizing the text at the word level. You can find the documentation [here](https://www.nltk.org/api/nltk.tokenize.html). Most tokenizers use either regular expressions or a machine learning model that was trained on a large dataset to learn token-splitting rules. 

TODO 4: Use word_tokenize() to tokenize each of the sentences from the last cell.

In [18]:
tokenized_sents = []

for sent in clean_sents:
    ### WRITE YOUR OWN CODE HERE
    tokens = nltk.word_tokenize(sent)
    #######
    
    print("<TOKENS>")
    print(tokens)
    
    tokenized_sents.append(tokens)

<TOKENS>
['Remembering', 'the', 'dirty', 'particulars', 'of', 'this', 'insidiously', 'vapid', '``', 'movie', "''", 'is', 'akin', 'to', 'digging', 'into', 'your', 'chest', 'cavity', 'with', 'a', 'rusty', ',', 'salted', 'spoon', '.']
<TOKENS>
['Perhaps', '``', 'Home', 'Alone', '2', ':', 'Lost', 'in', 'New', 'York', "''", '(', '1992', ')', 'was', 'a', 'bit', 'on', 'the', 'predictable', 'side', ',', 'but', 'this', 'pathetic', 'excuse', 'for', 'a', 'film', 'is', 'just', 'one', 'of', 'the', 'most', 'shameless', 'bids', 'at', 'commercialization', 'I', 'have', 'ever', 'heard', 'of', '.']
<TOKENS>
['A', 'boy', 'fighting', 'off', 'spies/terrorists', 'when', 'he', "'s", 'home', 'alone', 'in', 'a', 'Chicago', 'suburb', 'with', 'the', 'chickenpox', '?']
<TOKENS>
['Ridiculous', '!']
<TOKENS>
['Why', 'did', 'this', 'film', 'have', 'to', 'be', 'made', '?']


Run the code below to see how NLTK has handled the non-letter characters. 
* What does it do with most punctuation marks? 
* When does it not split tokens based on punctuation?

In [19]:
for sent in tokenized_sents:
    for tok in sent:
        if re.search(r'[^a-zA-Z0-9]', tok):  # find the non-letter and non-digit characters
            print(tok)  # print the entire token containing the non-letter/non-digit character

``
''
,
.
``
:
''
(
)
,
.
spies/terrorists
's
?
!
?


## 3.3 Word Normalisation

Many words can appear in different forms, including: 
* Conjugated verbs
* Plural and singular nouns
* Common abbrevations and synonyms like "USA" and "US". 

Mapping all of these surface forms to a single root form reduces the size of the vocabulary that we have to deal with and can therefore improve the performance of text classifiers or topic models.

The two most widely used tools for this task in English are the Porter Stemmer and WordNet Lemmatizer. These tools apply a series of regular expression substitutions to tokenised text to convert words to a standard format. 
* The Porter stemmer is much faster but just removes word prefixes and endings, which leads to some errors. It is often used when real-time or high-volume text processing is needed.
* As well as applying regular expressions, lemmatizers look words up in a dictionary to find their root forms, so are more accurate but much slower. 

Let's start by applying the [Porter Stemmer class](https://www.nltk.org/_modules/nltk/stem/porter.html) to our tokenised text by calling the stem() method:

In [20]:
stemmer = nltk.PorterStemmer() 
stemmed_sents = []

for sent in tokenized_sents:
    stemmed_sent = [stemmer.stem(tok) for tok in sent]
    
    stemmed_sents.append(stemmed_sent)
    
    print("<STEMMED TOKENS>")
    print(stemmed_sent)

<STEMMED TOKENS>
['rememb', 'the', 'dirti', 'particular', 'of', 'thi', 'insidi', 'vapid', '``', 'movi', "''", 'is', 'akin', 'to', 'dig', 'into', 'your', 'chest', 'caviti', 'with', 'a', 'rusti', ',', 'salt', 'spoon', '.']
<STEMMED TOKENS>
['perhap', '``', 'home', 'alon', '2', ':', 'lost', 'in', 'new', 'york', "''", '(', '1992', ')', 'wa', 'a', 'bit', 'on', 'the', 'predict', 'side', ',', 'but', 'thi', 'pathet', 'excus', 'for', 'a', 'film', 'is', 'just', 'one', 'of', 'the', 'most', 'shameless', 'bid', 'at', 'commerci', 'i', 'have', 'ever', 'heard', 'of', '.']
<STEMMED TOKENS>
['a', 'boy', 'fight', 'off', 'spies/terrorist', 'when', 'he', "'s", 'home', 'alon', 'in', 'a', 'chicago', 'suburb', 'with', 'the', 'chickenpox', '?']
<STEMMED TOKENS>
['ridicul', '!']
<STEMMED TOKENS>
['whi', 'did', 'thi', 'film', 'have', 'to', 'be', 'made', '?']


Now let's compare the stemming results to lemmatisation. For this task, NLTK provides the [class WordNetLemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html) with the method lemmatize(). This method takes an argument, `pos`, that determines whether the lemmatizer is applied to nouns, verbs, adjectives or adverbs.

TODO 5: Use the WordNetLemmatizer to lemmatize the nouns in the tokenized sentences. Set the `pos` argument to 'n'. 

TODO 6: Add a second call to lemmatize() to lemmatize the verbs in the sentences as well. Set the `pos` argument to 'v'. 

How do the results compare with the Porter stemmer? 

How have the verbs in the sentences changed?

In [21]:
lemmatizer = nltk.WordNetLemmatizer() 
lemma_sents = []
for sent in tokenized_sents:
    
    ### WRITE YOUR OWN CODE HERE
    lemma_sent = [lemmatizer.lemmatize(word, pos= 'n') for word in sent]
    lemma_sent = [lemmatizer.lemmatize(word, pos= 'v') for word in lemma_sent]
    #######
    
    lemma_sents.append(lemma_sent)
    
    print("<LEMMATIZED TOKENS>")
    print(lemma_sent)

<LEMMATIZED TOKENS>
['Remembering', 'the', 'dirty', 'particular', 'of', 'this', 'insidiously', 'vapid', '``', 'movie', "''", 'be', 'akin', 'to', 'dig', 'into', 'your', 'chest', 'cavity', 'with', 'a', 'rusty', ',', 'salt', 'spoon', '.']
<LEMMATIZED TOKENS>
['Perhaps', '``', 'Home', 'Alone', '2', ':', 'Lost', 'in', 'New', 'York', "''", '(', '1992', ')', 'wa', 'a', 'bite', 'on', 'the', 'predictable', 'side', ',', 'but', 'this', 'pathetic', 'excuse', 'for', 'a', 'film', 'be', 'just', 'one', 'of', 'the', 'most', 'shameless', 'bid', 'at', 'commercialization', 'I', 'have', 'ever', 'hear', 'of', '.']
<LEMMATIZED TOKENS>
['A', 'boy', 'fight', 'off', 'spies/terrorists', 'when', 'he', "'s", 'home', 'alone', 'in', 'a', 'Chicago', 'suburb', 'with', 'the', 'chickenpox', '?']
<LEMMATIZED TOKENS>
['Ridiculous', '!']
<LEMMATIZED TOKENS>
['Why', 'do', 'this', 'film', 'have', 'to', 'be', 'make', '?']


# 4. Classification

In this part, we will introduce text processing tools from the scikit-learn library. Let's start by loading the test set for the IMDB sentiment dataset.

## 4.1 Preprocessing


In [22]:
# Load the test set:
test_dataset = load_dataset(
    "imdb",
    split="test",
    cache_dir=cache_dir,
)
print(f"Test dataset with {len(test_dataset)} instances loaded")

test_dataset = np.random.choice(test_dataset, 2000, replace=False)  # we'll only use a subset of the data in this lab so that the code runs quicker


Test dataset with 25000 instances loaded


The next step is to tokenise the text of each tweet and convert it to a bag of words, ready for input to a classifier. 
To do this, we will use the scikit-learn library. 

To extract a bag of words, we can use the CountVectorizer class ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)).
This class outputs the bag of words as a feature vector, where the length of the vector is equal to the size of the vocabulary, and the values are the counts of each words in a document. 

Run the code below to obtain feature vectors for the training and test samples:

In [23]:
train_text = [sample["text"] for sample in train_dataset]
train_label = [sample["label"] for sample in train_dataset]
test_text = [sample["text"] for sample in test_dataset]
test_label =  [sample["label"] for sample in test_dataset]

In [24]:
train_text

['Remembering the dirty particulars of this insidiously vapid "movie" is akin to digging into your chest cavity with a rusty, salted spoon. Perhaps "Home Alone 2: Lost in New York" (1992) was a bit on the predictable side, but this pathetic excuse for a film is just one of the most shameless bids at commercialization I have ever heard of. A boy fighting off spies/terrorists when he\'s home alone in a Chicago suburb with the chickenpox? Ridiculous! Why did this film have to be made? I am the kind of person who believes even terrible movies are not wastes of time, but rather learning experiences. However, this is actually a waste of time. It should be avoided at all costs.',
 'Gosha\'s last great film of the 1960\'s. A resolute stylist with a great sense of purpose to his films, Gosha teamed up with Shintaro Katsu (of Zatoichi fame) to produce this scathing indictment of mindless nationalist loyalty. <br /><br />"Tenchu" (heavenly judgment) is the word that the loyalists to the emperor y

In [25]:
set(train_label)

{0, 1}

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

# CountVectorizer can do its own tokenization, but for consistency we want to
# carry on using WordNetTokenizer. 
vectorizer = CountVectorizer(tokenizer=word_tokenize)  # construct the vectorizer

vectorizer.fit(train_text)  # Learn the vocabulary
X_train = vectorizer.transform(train_text)  # extract training set bags of words
X_test = vectorizer.transform(test_text)  # extract test set bags of words

c:\Users\Ha Tung Lam\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


The fit() method sets the vectorizer up by extracting a vocabulary from some text data. 

QUESTION: Why do we fit the CountVectorizer on the training set only?

The vectorizer stores the vocabulary as a dictionary that maps a token to its index in the feature vector. The code below looks up the indexes of some example words:

In [27]:
import reprlib

vocabulary = vectorizer.vocabulary_
print(vocabulary['the'])
print(vocabulary['horse'])
print(vocabulary['smile'])

print(f'Vocabulary size = {len(vocabulary)}')

43440
21595
40084
Vocabulary size = 48762


In [28]:
X_train

<5000x48762 sparse matrix of type '<class 'numpy.int64'>'
	with 740742 stored elements in Compressed Sparse Row format>

## 4.2 Naive Bayes Classifier

The code above has obtained the feature vectors and lists of labels. The data is now ready for use
with scikit-learn's classifiers.

Scikit-learn contains several different variants of naïve Bayes for different kinds of data. For our bag of words data, we need to use the [MultinomialNB class](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB).


TODO 7: Look at the documentation for MultinomalNB and write code to train a NB classifier using `X_train` and `train_label`.

In [29]:
# WRITE YOUR CODE HERE
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

nb.fit(X_train, train_label)


MultinomialNB()

Now we have a trained model, we would like to evaluate its performance on some test data. 

TODO 8: Refer to the documentation again and predict the labels for the test set. Use `X_test` as the inputs to the classifier.

In [30]:
# WRITE YOUR CODE HERE
nb_pred = nb.predict(X_test)


We can compute metrics for classifier performance using [scikit-learn's metrics libary](https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules). A useful function for multi-class classification (when there are more than two classes) is the [classification report function](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-report).

TODO 9: Refer again to the documentation, and compute accuracy, precision, recall and F1 scores on the test set. 

In [31]:
# WRITE YOUR CODE HERE
from sklearn.metrics import classification_report
report = classification_report(test_label, nb_pred)
print(report)

              precision    recall  f1-score   support

           0       0.76      0.87      0.81      1004
           1       0.85      0.72      0.78       996

    accuracy                           0.79      2000
   macro avg       0.80      0.79      0.79      2000
weighted avg       0.80      0.79      0.79      2000



Now, let's examine the classifier that we learned. You may wish to refer back to the slides on naïve Bayes classifiers or to [Jurafsky and Martin's textbook](https://web.stanford.edu/~jurafsky/slp3/4.pdf) to see how the implementation relates to the equations. 

Previously, we trained a MultinomialNB classifier. The trained classifier object stores all the probabilities that it learned during training, which are needed to make predictions. The log of the likelihoods of each word given the class are represented by the attribute `feature_log_prob_`. So, if your classifier object is named `classifier`, you can access the log likelihoods with `classifier.feature_log_prob_`.

TODO 10: Print out the likelihood of the words 'happy' and 'hate' in each class. Hint: look up the index of the chosen words in `vocabulary`. The rows of `feature_log_prob` correspond to classes, and the columns to words.

In [32]:
import numpy as np

### CHANGE THE NAME OF THE CLASSIFIER VARIABLE BELOW TO USE YOUR TRAINED CLASSIFIER
feat_likelihoods = np.exp(nb.feature_log_prob_)  # Use exponential to convert the logs back to probabilities
###

# WRITE YOUR CODE HERE
feat_likelihoods[:, vocabulary['happy']], feat_likelihoods[:, vocabulary['hate']] 

(array([9.32648148e-05, 1.39057929e-04]),
 array([1.09253069e-04, 9.66783696e-05]))

The sentiment classes are negative (0) and positive (1). 

QUESTION: According to the feature log likelihoods, which class has the strongest association with 'happy' and with 'hate'?

A key part of evaluating a classifier is investigating the errors it makes to better understand its limitations. 

TODO 11: Complete the code below to print out some misclassified reviews along with their predicted and true labels. Can you see any reasons why a NB classiier would make mistakes with these instances?

In [33]:
error_indexes = nb_pred != test_label  # compare predictions to gold labels

# get the text of reviews where the classifier made an error:
reviews_err = np.array(test_text)[error_indexes]

# WRITE YOUR CODE HERE
print('Review: ', reviews_err[2])
print('Predicted label: ', nb_pred[error_indexes][2])
print('Actual label: ', np.array(test_label)[error_indexes][2])



Review:  I just can't understand why people are surprised this movie makes no sense. It was never supposed to make sense. (Duh! The writers were completely wasted on Frodis at the time.) It was just supposed to entertain and mock, and it does both wonderfully.<br /><br />The Monkees are good actors. They wouldn't have been hired if they weren't good actors. Mike has a thing for deadpan and darkness, Micky is the best at sheer psychotic comedy, Davy is a Broadway veteran, and Peter actually had people believing he was that dumb in real life. Don't tell me they can't act, because they most definitely can.<br /><br />They can also write. Sure, Jack & Bob get the sole credits, but in reality, they got a big helping hand on that script from the Monkees, who were also in that smoke-filled room.<br /><br />(it is absolutely impossible to spoil this film) Head is very highly symbolic. Among the more memorable elements is the black box, which was actually based on 2 things: the Monkee image tha

## 4.3. Logistic Regression Classifier

Another simple, linear classifier is logistic regression. This classifier does not rely on the conditional independence assumption, so can better model features that are highly correlated with each other. Scikit-learn provides the [logisticRegression class](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), which has a very similar interface to the naïve Bayes classifier.

TODO 12: Train a logistic regression classifier, referring to the scikit-learn documentation as required.

In [34]:
# WRITE YOUR CODE HERE
from sklearn.linear_model import LogisticRegression

lgr = LogisticRegression()

lgr.fit(X_train, train_label)


c:\Users\Ha Tung Lam\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [35]:
# WRITE YOUR CODE HERE

log_pred = lgr.predict(X_test)

In [36]:
# WRITE YOUR CODE HERE
report_log = classification_report(test_label, log_pred)
print(report_log)

              precision    recall  f1-score   support

           0       0.86      0.84      0.85      1004
           1       0.84      0.86      0.85       996

    accuracy                           0.85      2000
   macro avg       0.85      0.85      0.85      2000
weighted avg       0.85      0.85      0.85      2000



QUESTION: How does the performance of logistic regression compare with naïve Bayes?

The logistic regression classifier works by learning a weight for each feature that indicates its importance in predicting a class. These weights are stored in the `coef_` attribute of the LogisticRegression object. For the binary case, it provides a single weight per feature. This allows it to compute the probability of the positive class as _p_, then compute the probability of the negative class as _1-p_.  For multi-class problems, there is a row of weights corresponding to each class, and columns corresponding to words in the vocabulary. 

TODO 13: Print out the weights for 'happy' and 'hate'.

In [37]:
### WRITE YOUR CODE HERE

lgr.coef_[:, vocabulary['happy']], lgr.coef_[:, vocabulary['hate']]

(array([0.07370557]), array([0.25749438]))

QUESTION: Are the weights what you would expect to see?

The code below prints out the words with the highest weights for the positive class. We use numpy's `argsort` function to get the indexes of the sorted weights. Run the code below to show the result: 

In [38]:
n_feats_to_show = 10

# Flip the index so that values are keys and keys are values:
keys = vectorizer.vocabulary_.values()
values = vectorizer.vocabulary_.keys()
vocab_inverted = dict(zip(keys, values))

strongest_idxs = np.argsort(lgr.coef_[0])[-n_feats_to_show:]

for idx in strongest_idxs:
    print(f'{vocab_inverted[idx]} with weight {lgr.coef_[0][idx]}')

job with weight 0.953895470407712
fun with weight 0.9592499755014906
along with weight 0.9597897506474131
favorite with weight 0.9724842525206123
beautiful with weight 1.0321093454300119
7 with weight 1.0358382178238428
highly with weight 1.0918873132297342
perfect with weight 1.1279417428875709
amazing with weight 1.2807212792556837
excellent with weight 1.4606203021810575


TODO 14: Use the same code as for naïve Bayes to print out examples of misclassified tweets and their labels. Hint: you should be able to copy and paste your code from above :) 

In [39]:
error_indexes = log_pred != test_label  # compare predictions to gold labels

# get the text of tweets where the classifier made an error:
reviews_err = np.array(test_text)[error_indexes]

# WRITE YOUR CODE HERE
print('Review: ', reviews_err[2])
print('Predicted label: ', log_pred[error_indexes][2])
print('Actual label: ', np.array(test_label)[error_indexes][2])



Review:  This disaster flick is a remake of a 1973 movie of the same title, based on a novel by Sakyo Komatsu. Japan is located right alongside the Pacific Ring of Fire (active volcanoes) and also along the edges of plate tectonics, whose shifting will cause earthquakes and tsunamis (a Japanese term in itself for tidal wave). Naturally, this makes a natural premise for a disaster picture, what with Hollywood having a field day with films like Armageddon, Deep Impact, and more recently, The Day After Tomorrow, which tackles how global warming becomes the catalyst for natural disasters gone bonkers around the world.<br /><br />But I'll have to say this: The Sinking of Japan makes all the films mentioned earlier, look like classics. This disaster movie IS a disaster, and a massive one at that. Having to look at my watch every 10-15 minutes is a signal that the movie doesn't engage, and feels than it had over-clocked its runtime.<br /><br />The special effects are gorgeous to look at. From

# 5. OPTIONAL: Lemmatization and N-grams

You only need to do this section if you finish the previous sections before the end of the lab.

In the previous lab, we tried out lemmatization. This is useful for reducing the size of the vocabulary. Could it help us here?

To apply lemmatization, we have to go back to the CountVectorizer and define a new tokenizer class that will carry out the extra step of lemmatization. Run the code below to test this out:

In [40]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

class LemmaTokenizer(object):  # this 'tokenizer' will also do additional preprocessing steps, namely, lemmatize verbs and adjectives
    
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        
    def __call__(self, tweets):
        return [self.wnl.lemmatize(self.wnl.lemmatize(tok, pos='v'), pos='a') for tok in word_tokenize(tweets)]
    
vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())

vectorizer.fit(train_text)
X_train = vectorizer.transform(train_text)
X_test = vectorizer.transform(test_text)

# Print out some of the features in the vocabulary:
print(list(vectorizer.vocabulary_)[:20])

c:\Users\Ha Tung Lam\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['remember', 'the', 'dirty', 'particulars', 'of', 'this', 'insidiously', 'vapid', '``', 'movie', "''", 'be', 'akin', 'to', 'dig', 'into', 'your', 'chest', 'cavity', 'with']


In [41]:
print(f'Vocabulary size: {len(vectorizer.vocabulary_)}')

Vocabulary size: 42094


TODO 15: Now, repeat your training of the NB classifier using the new features, and compare its performance with the previous classifers.

In [51]:
### WRITE YOUR OWN CODE HERE

nb2 = MultinomialNB()
nb2.fit(X_train, train_label)

MultinomialNB()

In [52]:
nb2_pred = nb2.predict(X_test)

In [54]:
report2 = classification_report(test_label, nb2_pred)
print(report2)

              precision    recall  f1-score   support

           0       0.76      0.88      0.81      1004
           1       0.85      0.71      0.78       996

    accuracy                           0.80      2000
   macro avg       0.81      0.80      0.80      2000
weighted avg       0.80      0.80      0.80      2000



In [55]:
print(report)

              precision    recall  f1-score   support

           0       0.76      0.87      0.81      1004
           1       0.85      0.72      0.78       996

    accuracy                           0.79      2000
   macro avg       0.80      0.79      0.79      2000
weighted avg       0.80      0.79      0.79      2000



QUESTION: Did lemmatization bring about any improvements on this dataset?

The bag of words is a very simple representation of the text that does not capture enough information to make accurate sentiment classifications. Another way to improve it could be to use bigrams instead of single words as our features. Bigrams are pairs of words that occur one after another in the text. Bigrams are a kind of 'n-gram', where 'n=2'. 

To extract bigrams, we again modify our CountVectorizer. This class has a parameter `ngram_range`, which determines the range of sizes of n-grams the vectorizer will include. If we set `ngram_range=(1,1)` we have our standard bag of words. If we set it to `ngram_range=(2,2)`, we use bigrams instead. Choosing If we set `ngram_range=(1,2)` will use both single tokens (unigrams) and bigrams.

TODO 16: Create a new CountVectorizer that extracts bigram features as well as unigrams (single tokens).

In [43]:
### WRITE YOUR CODE HERE



###

print(f'Vocabulary size: {len(vectorizer.vocabulary_)}')

# Print out some of the features in the vocabulary:
print(list(vectorizer.vocabulary_)[:20])

Vocabulary size: 42094
['remember', 'the', 'dirty', 'particulars', 'of', 'this', 'insidiously', 'vapid', '``', 'movie', "''", 'be', 'akin', 'to', 'dig', 'into', 'your', 'chest', 'cavity', 'with']


TODO 17: Now, repeat your training of the logistic regression or naïve Bayes classifier using the new features, and compare its performance with the previous classifers.

In [44]:
### WRITE YOUR OWN CODE HERE



QUESTION: Do bigrams improve performance on this dataset?

# 6. Optional: Lexicon Features

You only need to do this part if you finish the other parts before the end of the lab session. 

The NLTK library contains sentiment lexicons, which are lists of words with negative or positive connotations. 

In [45]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyser = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to C:\Users\Ha Tung
[nltk_data]     Lam\AppData\Roaming\nltk_data...


Let's have a look at the sentiment scores for some words in the lexicon by running the code below. What do the scores mean and why do some words have no score?

In [46]:
testwords = ['happy', 'wonderful', 'horrible', 'boring', 'tablecloth', 'not']

for word in testwords:
    if word in analyser.lexicon:
        print(f'{word}: {analyser.lexicon[word]}')
    else:
        print(f'{word}: NOT IN LEXICON')

happy: 2.7
wonderful: 2.7
horrible: -2.5
boring: -1.3
tablecloth: NOT IN LEXICON
not: NOT IN LEXICON


Now we would like to use this function to compute counts of all positive and negative words. Let's start by recording whether the words in our vocabulary are positive or negative:

In [47]:
# Print out some of the features in the vocabulary:
print(list(vectorizer.vocabulary_)[:20])

vocabulary = vectorizer.vocabulary_

lex_pos_scores = np.zeros((1, len(vocabulary)))
lex_neg_scores = np.zeros((1, len(vocabulary)))

# get the Vader lexicon scores for each word in our vocabulary
for i, term in enumerate(vocabulary):
    if term in analyser.lexicon and analyser.lexicon[term] > 0:
        lex_pos_scores[0, i] = 1
    elif term in analyser.lexicon and analyser.lexicon[term] < 0:
        lex_neg_scores[0, i] = 1

['remember', 'the', 'dirty', 'particulars', 'of', 'this', 'insidiously', 'vapid', '``', 'movie', "''", 'be', 'akin', 'to', 'dig', 'into', 'your', 'chest', 'cavity', 'with']


Now let's compute the counts of positive and negative words in the dataset:

In [48]:
# Compute the scores for each instance in the data set. 

# Multiply the lexicon scores by the feature vectors, then sum over the 
# vocabulary to get the total positive and total negative counts:
lex_pos_train = np.sum(X_train.multiply(lex_pos_scores), axis=1)
lex_pos_test = np.sum(X_test.multiply(lex_pos_scores), axis=1)

lex_neg_train = np.sum(X_train.multiply(lex_neg_scores), axis=1)
lex_neg_test = np.sum(X_test.multiply(lex_neg_scores), axis=1)

Finally, we can append the counts to the feature vector and treat them as extra features:

In [49]:
from scipy.sparse import hstack

X_train = hstack((X_train, lex_pos_train, lex_neg_train))
X_test = hstack((X_test, lex_pos_test, lex_neg_test))

TO-DO 18: Use the new X_train and X_test feature vectors to train and evaluate your classifier. 
Does adding the lexicon features improve performance?

In [50]:
### WRITE YOUR OWN CODE HERE

